<a href="https://colab.research.google.com/github/ericburdett/named-entity-recognition/blob/master/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Named Entity Recognition

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import pdb
import torchvision
import os
import gzip
import tarfile
from PIL import Image, ImageOps
import gc
import pdb
from google.colab import drive
drive.mount('/content/drive')
from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

assert torch.cuda.is_available(), "Request a GPU from Runtime > Change Runtime"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r "drive/My Drive/datasets/twitter-ner/" "/content"

In [0]:
import pandas as pd
data = pd.read_csv('/content/twitter-ner/a.conll.txt', sep='\t', header=None, names=['input', 'output'])
data = pd.concat((data, pd.read_csv('/content/twitter-ner/b.conll.txt', sep='\t', header=None, names=['input', 'output'])))
data

,input,output
0,I,O
1,hate,O
2,the,O
3,words,O
4,chunder,O
...,...,...
30820,sitting,O
30821,on,O
30822,my,O
30823,piano,O


In [0]:
class twitter_dataset(Dataset):
  def __init__(self, dataset_type='train'):
    if not os.path.exists('/content/twitter-ner/'):
      raise Exception('Twitter-ner dataset does not exist!')
    
    if dataset_type == 'train':
      self.dataset = load_csv('a.conll')
      self.dataset = pd.concat((self.dataset, load_csv('b.conll')))
      self.dataset = pd.concat((self.dataset, load_csv('e.conll')))
      self.dataset = pd.concat((self.dataset, load_csv('f.conll')))

    else:
      self.dataset = load_csv('g.conll')
      self.dataset = pd.concat((self.dataset, load_csv('h.conll')))

  def __getitem__(self, index):
    return (self.dataset['input'][index], self.dataset['output'][index])

  def __len__(self):
    return len(self.dataset)

  def load_csv(filename):
    return pd.read_csv('/content/twitter/ner/' + filename, sep='\t', header=None, names=['input', 'target'])

In [0]:
dataset = twitter_dataset(dataset_type='train')
dataset[0]

NameError: ignored